# Beispiel: SimplePaint

<div class="prereq">
    <h3>Was man wissen sollte</h3>
    <div>
        <a class="prereq" href="/user-redirect/algoviz/lessons/03_Fortgeschritten/02_MultArray.ipynb">
            Mehrdimensionale Arrays</a>
    </div>
</div>

Im Folgenden werden wir ein Beiepiel vorführen, in dem ein Bild durch Mausklicks gemalt wird. Dazu führen wir die Möglichkeit ein auf Mausklicks zu reagieren. Das Ganze funktioniert ähnlich wie das Verwenden von Tasten.

## Die Maus

Als erstes benötigen Wir ein SVG in das wir klicken können.

In [1]:
#include <algoviz/SVG.hpp>
using namespace std;

AlgoViz::clear();

SVG zeichnung = SVG(400,400);

Ob es einen Mausclick gab, fragen wir mit Die Maus fragen wir `zeichnung.lastClick()` ab. Dieser Befehl gibt als Ergebnis ein Objekt der Klasse `MausState` zurück. In ihm werden dei Koordinaten des Mausklicks und die gedrückten Tasten gespeichert.

Noch eine Bemerkung: Mit `AlgoViz::sendText("Text");` kann man eine Nachricht im Sidebar anzeigen lassen.

In [2]:
// Die Variable zum Speichern der Daten das Klicks
MouseState maus;
string taste;

// Gebe eine Nachricht aus, die den Beginn der Schleife anzeigt.
AlgoViz::sendText("Beenden mit beliebiger Taste");

do {
    taste = zeichnung.lastKey();
    
    // Hole den letzten Klick
    maus = zeichnung.lastClick();
    
    // Wenn es einen Klick gab ...
    if ( maus.isLegal() ) {
        // ... gebe die Koordinaten des Klicks im SVG aus und welche Taste gedrückt wurde.
        cout << maus.x() << "," << maus.y() << endl;
    }
  
} while ( taste == "" );

// Gebe eine Nachricht aus, die das Ende anzeigt.
AlgoViz::sendText("Ende");

167,76
199,167
314,281
75,283
46,128
227,81
342,303
208,393
45,69
325,37
364,-8
386,175
262,353
55,393
361,411
13,-19
253,9
358,-17
305,168
137,180
71,281
163,337
280,261
164,330
89,150
179,197
102,49
255,45
302,95


Wie man sieht kann man mit `maus.x()`und `maus.y()` die Koordinaten des Klicks im SVG abfragen. Und das werden wir nutzen, um ein Bild zu zeichnen.

## Die Pixel

Als erstes benötigen wir Pixel, d.h. Bildpunkte, die wir mit dem Mausklick einfärben können. Jedes Pixel ist ein Rechteck und sie sind alle in einm zweidimensionalen Gitter angeordnet. Daher eignet sich eine zweidimensionales Array `rects`, in der wir Objekte der lasse `Rect` verwalten. Das Ganze funktioniert so ähnlich, wie beim Beispiel [Snake](/user-redirect/algoviz/lessons/03_Fortgeschritten/01_Snake.ipynb). Außerdem speichern wir in einem weiteren zweidimensionalen Array `bild` für jeden Pixel die Farbe, 0 für weiß und 1 für schwarz.

In [3]:
/**
 ** Das Gitter aus Rechtecken.
 **/
#include <algoviz/SVG.hpp>
using namespace std;

AlgoViz::clear();

SVG zeichnung = SVG(400,400);

// Die "Farbwerte" der Pixel.
// Das Bild besteht aus 40x40 Pixeln.
int bild[40][40];

// Die Rechtecke, die die Pixel darstellen.
Rect rects[40][40];

// Durchlaufe alle Pixel
for ( int spalte = 0 ; spalte < 40; spalte = spalte+1 ) {
    for ( int zeile = 0 ; zeile < 40; zeile = zeile+1 ) {
        
        // Bewege das Rechteck an die richtige Stelle
        rects[spalte][zeile].moveTo(spalte*10,zeile*10);
        // Setze es auf die richtige Größe
        rects[spalte][zeile].setWidth(10);
        rects[spalte][zeile].setHeight(10);
    
        // Alle Pixel solen weiß sein.
        bild[spalte][zeile] = 0;
        
        // Setze die Farbe der Pixel (in diesem Beispiel eigentlich unnötig)
        if ( bild[spalte][zeile] == 1 ) {
            rects[spalte][zeile].setFill("black");
        } else {
            rects[spalte][zeile].setFill("white");
        }

        // Füge das Rechteck zur Zeichnung hinzu.
        rects[spalte][zeile].addTo(&zeichnung);
        
    }    
}

Das man 1600 Rechtecke zeichnet und die Kommunikation etwas langsam ist, sieht man den Bildaufbau. Insbesondere sieht man, dass das Gitter spaltenweise aufgebaut wird, da die äußere Schleife die Spalten durchläuft.

## Das "Malen"

Jetzt verwenden wir die Maus um die Pixel zu färben. Dazu bestimmen wir die Koordinaten des Mausklicks. Da die Piexl eine Breite und Höhe von 10 haben, müssen wir die Koordinaten des Klicks jeweils durch 10 teilen, um die Koordinaten des Pixels zu erhalten. Dann können wir seinen Wert "kippen" und seine Farbe aktualisieren.

In [4]:
/**
 ** Jeder Mausklick verändert die Farbe des jeweiligen Pixels
 **/
string taste = "";
MouseState maus;

AlgoViz::sendText("Beenden mit einer beliebigen Taste");

do {
    // Hole die letzte Taste
    taste = zeichnung.lastKey();
    
    // Hole den letzten Mausklick
    maus = zeichnung.lastClick();
    
    // Falls er "legal" ist, d.h. seine Koordinaten im SVG liegen ...
    if ( maus.isLegal() ) {
        // Berechne die Koordinaten des Pixels
        int spalte = maus.x()/10;
        int zeile = maus.y()/10;
    
        // "Kippe" den Wert des Pixels, 0 -> 1 -> 0
        bild[spalte][zeile] = 1- bild[spalte][zeile];
        
        // Aktualisiere die Farbe des Pixels.
        if ( bild[spalte][zeile] == 1 ) {
            rects[spalte][zeile].setFill("black");
        } else {
            rects[spalte][zeile].setFill("white");
        }
    }
    
} while ( taste == "" ); // Fahre fort, wenn keine Taste gedrückt wurde.

AlgoViz::sendText("Ende!");

## Speichern

Jetzt kann man das Bild speichern. Dazu verwenden wir ein einfaches System mit dem man zweifarbige Bilder komprimieren kann. Ein ähnliches System wird bei der Übertragung von Faxen eingesetzt.

Hier ist der Programmcode, der die Kompression durchführt.

In [21]:
int aktuell = 0;
int zaehler = 0;
string daten = "";

for ( int zeile = 0 ; zeile < 40; zeile = zeile+1 ) {
    for ( int spalte = 0 ; spalte < 40; spalte = spalte+1 ) {
        if ( bild[spalte][zeile] != aktuell ) {
            daten = daten + to_string(zaehler) + " ";
            aktuell = bild[spalte][zeile];
            zaehler = 1;
        } else {
            zaehler=zaehler+1;
        }
    }    
}

daten = daten + to_string(zaehler);

cout << daten;

492 1 50 1 74 1 40 1 37 1 42 1 43 1 124 1 195 1 27 1 466

<div class="task">
    <h3>Aufgabe</h3>
    <div>
        Die folgende Sequenz codiert ein Bild der Breite 12 Pixel. Rekonstruieren Sie das Bild (von Hand oder mit Programm).
        <center>
            <a title="all" target="_new" href="https://www.youtube.com/watch?v=Gsy5sJy5_34">
                1 2 9 1 2 1 6 1 1 1 2 1 5 1 1 2 2 1 5 1 1 2 2 3 3 1 1 2 2 1</a><br/>
            <a title="over"  target="_new" href="https://www.youtube.com/watch?v=CbI79e5iZKs">
                2 4 1 2 2 1 2 1 2 1 1 6 1 1 2 1 1 2 4 2 2 1 1 2 4 2 2 1 1</a><br/>
            <a title="the world"  target="_new" href="https://www.youtube.com/watch?v=v4xZUr0BEfE">
                2 2 2 2 2 2 2 2 1 7 2 3 1 6 2 4 1 5 2 4 1 5 1 1 1 8 1 3 8 2</a>
            </center></a>
    </div>
</div>                

In [17]:
int code[] = {
    1, 2, 9, 1, 2, 1, 6, 1, 1, 1, 2, 1, 5, 1, 1, 2, 2, 1, 5, 1,
    1, 2, 2, 3, 3, 1, 1, 2, 2, 1, 2, 4, 1, 2, 2, 1, 2, 1, 2, 1,
    1, 6, 1, 1, 2, 1, 1, 2, 4, 2, 2, 1, 1, 2, 4, 2, 2, 1, 1, 2,
    2, 2, 2, 2, 2, 2, 2, 1, 7, 2, 3, 1, 6, 2, 4, 1, 5, 2, 4, 1,
    5, 1, 1, 1, 8, 1, 3, 8, 2 };

<div class="followup">
    <h3>Wo es weiter geht</h3>
    <div>
        <a class="followup" href="/user-redirect/algoviz/lessons/03_Fortgeschritten/04_Vektoren.ipynb">Vektoren</a>.
    </div>
</div>